# Importações

In [1]:
import pandas as pd
import os
import re
from datetime import datetime

# Dados

## Carregando os dados

In [2]:
#diretorio_script = os.path.dirname(__file__)
#for file in os.listdir(os.path.join(diretorio_script, f"data/{pasta}")):

df_dict = dict()
for file in os.listdir("../data"):
    if "xlsx" in file:
        df_dict.update({f"{file}": pd.read_excel(f"../data/{file}")})

In [3]:
for key in df_dict:
    print(f'key: {key}, dataframe: {len(df_dict[key].columns)}')

key: 2020_1.Bolsa_pernamencia.xlsx, dataframe: 15
key: 2022_2.Bolsa_permanencia.xlsx, dataframe: 54
key: Cópia de 2019_1.Bolsa_permanencia.xlsx, dataframe: 47
key: Cópia de 2021_1.Bolsa_permanencia.xlsx, dataframe: 60
key: Cópia de 2021_2.Bolsa_permanencia.xlsx, dataframe: 60
key: Cópia de 2022_1.Bolsa_permanencia.xlsx, dataframe: 59


In [4]:
def find_missing_columns(dfs):
    # Cria conjuntos de colunas para cada DataFrame
    column_sets = [set(df.columns) for df in dfs]
    
    # Encontra a interseção das colunas
    common_cols = set.intersection(*column_sets)
    
    # Encontra a união de todas as colunas
    all_cols = set.union(*column_sets)
    
    # Identifica colunas que não estão presentes em todos os DataFrames
    missing_cols = all_cols - common_cols
    
    print("Colunas que não estão presentes em todos os DataFrames:")
    print(missing_cols)
    print("Número de colunas que não estão presentes em todos os DataFrames:", len(missing_cols))

In [5]:
df_list = list(df_dict.values())

In [6]:
find_missing_columns(df_list)

Colunas que não estão presentes em todos os DataFrames:
{'Indice', 'Renda bruta familiar', 'Condiçõees de moradia familiar', 'Auxílio acidente', 'PROIFPE', 'Sua família/você recebeu Auxílio Emergencial do governo?', 'Quantas pessoas compõem o seu núcleo familiar?', 'data inicio', 'Documentação correta?', 'Mudou de endereço para estudar no IFPE', 'Auxílio moradia', 'Quais as principais consequências da covid 19 para o núcleo familiar?', 'Acesso ao ensino fundamental', 'Algum membro familiar apresentou sintomas relativos a COVID 19 , ou testou positivo no exame para a doença?', 'Recebe algum tipo de bolsa estudantil', 'Valor recebido através do programa social (BPC ou Auxílio Brasil)', 'Moradia estudantil.1', 'Auxílio digital', 'Você possui filhos entre 0 e 6 anos', 'Foi beneficiado na última edição do Programa de Manutenção Acadêmica (antigo PBP)?', 'Foi beneficiado na última edição do Benefício Eventual', 'Material de construção da moradia', 'Auxílio reclusão', 'Moradia estudantil', 'S

Melhor combinação que eu encontrei:

In [7]:
find_missing_columns([df_list[1], df_list[3], df_list[4], df_list[5]])

Colunas que não estão presentes em todos os DataFrames:
{'Algum membro familiar apresentou sintomas relativos a COVID 19 , ou testou positivo no exame para a doença?', 'Indice', 'Valor recebido através do programa social (BPC ou Bolsa Família)', 'Valor recebido através do programa social (BPC ou Auxílio Brasil)', 'Sua família/você recebeu Auxílio Emergencial do governo?', 'Moradia estudantil.1', 'Em que medida a pandemia de Covid-19 afetou a renda da sua família?', 'process', 'O estudante já é participante do PBP', 'Qual foi valor mensal do auxílio emergencial?', 'Você está se deslocando para assistir às aulas presencialmente?', 'Quais as principais consequências da covid 19 para o núcleo familiar?', 'Benefício Eventual', 'Foi beneficiado na última edição do Programa de Manutenção Acadêmica (antigo PBP)?', 'Foi beneficiado na última edição do Benefício Eventual'}
Número de colunas que não estão presentes em todos os DataFrames: 15


In [8]:
df = pd.concat([df_list[1], df_list[3], df_list[4], df_list[5]], ignore_index=True)

In [9]:
del df_dict, df_list

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18800 entries, 0 to 18799
Data columns (total 64 columns):
 #   Column                                                                                                       Non-Null Count  Dtype         
---  ------                                                                                                       --------------  -----         
 0   Data de criação                                                                                              18800 non-null  object        
 1   Passo                                                                                                        18800 non-null  object        
 2   Nome do evento                                                                                               18800 non-null  object        
 3   created at                                                                                                   18800 non-null  object        
 4   process         

In [11]:
df

,Data de criação,Passo,Nome do evento,created at,process,Indice,Foi beneficiado na última edição do Programa de Manutenção Acadêmica (antigo PBP)?,Foi beneficiado na última edição do Benefício Eventual,Data de nascimento,Raça,...,O estudante já é participante do PBP,Benefício Eventual,Valor recebido através do programa social (BPC ou Bolsa Família),Moradia estudantil.1,Você está se deslocando para assistir às aulas presencialmente?,Em que medida a pandemia de Covid-19 afetou a renda da sua família?,"Algum membro familiar apresentou sintomas relativos a COVID 19 , ou testou positivo no exame para a doença?",Quais as principais consequências da covid 19 para o núcleo familiar?,Sua família/você recebeu Auxílio Emergencial do governo?,Qual foi valor mensal do auxílio emergencial?
0,2022-09-01 03:00:51.418005+00:00,['Concessão da bolsa'],2022-02-01 00:00:00,2022-09-01 03:00:51.418005+00:00,pma_manutencao_academica.process.ProgramaDeApo...,2.0,Não,Não,2003-01-29,Branca,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-01 03:06:15.743581+00:00,['Concessão da bolsa'],2022-02-01 00:00:00,2022-09-01 03:06:15.743581+00:00,pma_manutencao_academica.process.ProgramaDeApo...,5.0,Não,Não,2003-02-11,Parda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-01 03:07:35.579459+00:00,['Não contemplado'],2022-02-01 00:00:00,2022-09-01 03:07:35.579459+00:00,pma_manutencao_academica.process.ProgramaDeApo...,1.0,Não,Não,2002-10-27,Indígena,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-01 03:30:46.439575+00:00,['Complementar as informações do membro famili...,2022-02-01 00:00:00,2022-09-01 03:30:46.439575+00:00,pma_manutencao_academica.process.ProgramaDeApo...,1.0,Não,Não,1980-04-12,Parda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-01 03:37:29.326782+00:00,['Concessão da bolsa'],2022-02-01 00:00:00,2022-09-01 03:37:29.326782+00:00,pma_manutencao_academica.process.ProgramaDeApo...,3.0,Não,Não,2004-03-22,Parda,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18795,2022-04-07 02:24:13.791294+00:00,['Concessão da bolsa'],2022-01-01 00:00:00,2022-04-07 02:24:13.791294+00:00,NaN,NaN,NaN,NaN,1998-05-03,Branca,...,Sim,Não,1212.0,NaN,Sim,Afetou pouco (a renda diminuiu),Sim,NaN,Sim,300.0
18796,2022-04-07 02:30:12.208103+00:00,['Complementar as informações do membro famili...,2022-01-01 00:00:00,2022-04-07 02:30:12.208103+00:00,NaN,NaN,NaN,NaN,1972-10-04,Parda,...,Não,Não,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN
18797,2022-04-07 02:38:40.179380+00:00,['Calcular renda per capita'],2022-01-01 00:00:00,2022-04-07 02:38:40.179380+00:00,NaN,NaN,NaN,NaN,2002-02-03,Parda,...,Não,Sim,136.0,NaN,Sim,NaN,NaN,NaN,NaN,NaN
18798,2022-04-07 02:44:57.533284+00:00,['Calcular renda per capita'],2022-01-01 00:00:00,2022-04-07 02:44:57.533284+00:00,NaN,NaN,NaN,NaN,1996-06-21,Preta,...,Não,Não,NaN,NaN,Sim,NaN,NaN,NaN,NaN,NaN


## Tratando os dados

### Remoção de espaços vazios nos valores

In [12]:
def remove_initial_and_ending_spaces(name):
    regex = r'^(?:\s+)?(?P<gp>.+?)(?:\s+)?$'
    mo = re.search(regex, name)
    if mo is not None:
      return mo['gp']
    else:
      print(f'Deu erro em: {name}')
      return name

In [13]:
for col in df.columns:
    df = df.rename({col:remove_initial_and_ending_spaces(col)}, axis='columns')

### Remoção de Colunas

In [14]:
columns_white_list = [
    'Data de nascimento', 
    'Raça', 
    'Sexo', 
    'Orientação Sexual', 
    'Gênero',
    'Turno', 
    'Período', 
    'Quantidade de disciplinas no período',
    'Origem escolar', 
    'É cotista por renda inferior a 1,5 salário mínimo?',
    'Moradia estudantil', 
    'Em seu endereço atual, como você mora?',
    'Condiçõees de moradia familiar', 
    'Região da moradia',
    'Abastecimento de água', 
    'Material de construção da moradia',
    'Saneamento', 
    'Mudou de endereço para estudar no IFPE',
    'Quantas pessoas compõem o seu núcleo familiar?',
    'Você possui filhos maiores que 6 anos',
    'Você possui filhos entre 0 e 6 anos',
    'Recebe algum tipo de bolsa estudantil',
    'Você é chefe de família ou responsável pela própria subsistência?',
    'Qual o grau de parentesco da pessoa que chefia a família?',
    'Renda bruta familiar', 
    'Renda per capita',
    'Qual a origem da renda principal de sua família?',
    'Alimentação no ambiente escolar',
    'Como você acessa os serviços de saúde',
    'Como você acessou à educação básica (Ensino Fundamental)',
    'Como você acessa/acessou à educação básica (Ensino Médio)',
    'Documentação correta?',
    #'Você está se deslocando para assistir às aulas presencialmente?', # ESTA COLUNA DEVE SER COLOCADA NA LISTA NO FUTURO 
    'Aluno contemplado com bolsa?'
]

In [15]:
df = df[columns_white_list]

In [16]:
del columns_white_list

In [17]:
len(df.columns)

33

In [18]:
df.columns

Index(['Data de nascimento', 'Raça', 'Sexo', 'Orientação Sexual', 'Gênero',
       'Turno', 'Período', 'Quantidade de disciplinas no período',
       'Origem escolar', 'É cotista por renda inferior a 1,5 salário mínimo?',
       'Moradia estudantil', 'Em seu endereço atual, como você mora?',
       'Condiçõees de moradia familiar', 'Região da moradia',
       'Abastecimento de água', 'Material de construção da moradia',
       'Saneamento', 'Mudou de endereço para estudar no IFPE',
       'Quantas pessoas compõem o seu núcleo familiar?',
       'Você possui filhos maiores que 6 anos',
       'Você possui filhos entre 0 e 6 anos',
       'Recebe algum tipo de bolsa estudantil',
       'Você é chefe de família ou responsável pela própria subsistência?',
       'Qual o grau de parentesco da pessoa que chefia a família?',
       'Renda bruta familiar', 'Renda per capita',
       'Qual a origem da renda principal de sua família?',
       'Alimentação no ambiente escolar',
       'Como você 

### Removendo linhas duplicadas

In [19]:
df[df.duplicated()]

,Data de nascimento,Raça,Sexo,Orientação Sexual,Gênero,Turno,Período,Quantidade de disciplinas no período,Origem escolar,"É cotista por renda inferior a 1,5 salário mínimo?",...,Qual o grau de parentesco da pessoa que chefia a família?,Renda bruta familiar,Renda per capita,Qual a origem da renda principal de sua família?,Alimentação no ambiente escolar,Como você acessa os serviços de saúde,Como você acessou à educação básica (Ensino Fundamental),Como você acessa/acessou à educação básica (Ensino Médio),Documentação correta?,Aluno contemplado com bolsa?
2123,2004-08-10,Branca,Feminino,Heterossexual,Mulher,Integral,4.0,5.0,Escola pública,Não,...,Mãe,1121.00,280.25,Trabalho formal,0.00,Sistema Público de Saúde,Escola privada,Escola pública,Sim,Sim
3044,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3060,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3115,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3183,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18292,1998-03-17,Branca,Masculino,Heterossexual,Homem,Noite,2.0,6.0,Escola particular,Não,...,NaN,NaN,NaN,NaN,5.00,Sistema Público de Saúde,Escola privada,Escola privada,NaN,NaN
18493,2000-11-25,Parda,Feminino,Heterossexual,Mulher,Manhã,4.0,3.0,Escola pública,Sim,...,NaN,NaN,NaN,NaN,30.00,Sistema Público de Saúde,Escola privada,Escola pública,NaN,NaN
18499,2002-08-26,Parda,Masculino,Heterossexual,Homem,Noite,1.0,8.0,Escola pública,Não,...,NaN,NaN,NaN,NaN,27.00,Sistema Público de Saúde,Escola pública,Escola pública,NaN,NaN
18500,2000-11-25,Parda,Feminino,Heterossexual,Mulher,Manhã,4.0,3.0,Escola pública,Sim,...,NaN,NaN,NaN,NaN,30.00,Sistema Público de Saúde,Escola privada,Escola pública,NaN,NaN


In [20]:
# Descartando duplicadas
initial_len = df.shape[0]
df = df.drop_duplicates()
print(f'Tamanho inicial: {initial_len}, tamanho final {df.shape[0]} | Descartadas {initial_len - df.shape[0]} duplicadas')

Tamanho inicial: 18800, tamanho final 18322 | Descartadas 478 duplicadas


In [21]:
del initial_len

### Tratando colunas individualmente

#### Data de nascimento

In [22]:
df['Data de nascimento']

0       2003-01-29
1       2003-02-11
2       2002-10-27
3       1980-04-12
4       2004-03-22
           ...    
18795   1998-05-03
18796   1972-10-04
18797   2002-02-03
18798   1996-06-21
18799   1992-01-29
Name: Data de nascimento, Length: 18322, dtype: datetime64[ns]

In [23]:
df['Data de nascimento'] = (datetime.now() - df['Data de nascimento']).dt.days // 365

In [24]:
df['Data de nascimento'] = df['Data de nascimento'].astype(float)

In [25]:
df['Data de nascimento']

0        21.0
1        21.0
2        21.0
3        44.0
4        20.0
         ... 
18795    26.0
18796    51.0
18797    22.0
18798    28.0
18799    32.0
Name: Data de nascimento, Length: 18322, dtype: float64

### Transformando números string em float64

In [26]:
columns_to_float64 = [
    'Renda bruta familiar', 
    'Renda per capita',
    'Alimentação no ambiente escolar'
    ]

In [27]:
def convert_columns_to_float64(df, columns_to_float64):
    for column in columns_to_float64:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('float64')
    return df

In [28]:
df = convert_columns_to_float64(df, columns_to_float64)

In [29]:
df.dtypes

Data de nascimento                                                   float64
Raça                                                                  object
Sexo                                                                  object
Orientação Sexual                                                     object
Gênero                                                                object
Turno                                                                 object
Período                                                              float64
Quantidade de disciplinas no período                                 float64
Origem escolar                                                        object
É cotista por renda inferior a 1,5 salário mínimo?                    object
Moradia estudantil                                                    object
Em seu endereço atual, como você mora?                                object
Condiçõees de moradia familiar                                        object

### Removendo linhas com valores numéricos de colunas não-numéricas

In [30]:
#non_numeric_columns = [
#    'Como você acessou à educação básica (Ensino Fundamental)',
#    'Como você acessa/acessou à educação básica (Ensino Médio)'
#]

non_numeric_columns = df.select_dtypes(exclude=['number']).columns
non_numeric_columns

Index(['Raça', 'Sexo', 'Orientação Sexual', 'Gênero', 'Turno',
       'Origem escolar', 'É cotista por renda inferior a 1,5 salário mínimo?',
       'Moradia estudantil', 'Em seu endereço atual, como você mora?',
       'Condiçõees de moradia familiar', 'Região da moradia',
       'Abastecimento de água', 'Material de construção da moradia',
       'Saneamento', 'Mudou de endereço para estudar no IFPE',
       'Recebe algum tipo de bolsa estudantil',
       'Você é chefe de família ou responsável pela própria subsistência?',
       'Qual o grau de parentesco da pessoa que chefia a família?',
       'Qual a origem da renda principal de sua família?',
       'Como você acessa os serviços de saúde',
       'Como você acessou à educação básica (Ensino Fundamental)',
       'Como você acessa/acessou à educação básica (Ensino Médio)',
       'Documentação correta?', 'Aluno contemplado com bolsa?'],
      dtype='object')

In [31]:
def is_numeric(value):
    try:
        pd.to_numeric(value)
        return True
    except ValueError:
        return False

In [32]:
def remove_numerics_in_non_numerics(df, non_numeric_columns):
    mask = df[non_numeric_columns].map(is_numeric)
    rows_to_drop = mask.any(axis=1)
    df = df[~rows_to_drop]
    
    return df

In [33]:
df = remove_numerics_in_non_numerics(df, non_numeric_columns)

### Removendo linhas com valores não-numéricos em colunas numéricas

In [34]:
#numeric_columns = [
#    'Renda bruta familiar',
#    'Renda per capita'
#]

numeric_columns = df.select_dtypes(include=['number']).columns
numeric_columns

Index(['Data de nascimento', 'Período', 'Quantidade de disciplinas no período',
       'Quantas pessoas compõem o seu núcleo familiar?',
       'Você possui filhos maiores que 6 anos',
       'Você possui filhos entre 0 e 6 anos', 'Renda bruta familiar',
       'Renda per capita', 'Alimentação no ambiente escolar'],
      dtype='object')

In [35]:
def remove_non_numerics_in_numerics(df, numeric_columns):

    for column in numeric_columns:
        df = df[pd.to_numeric(df[column], errors='coerce').notna()]

    return df

In [36]:
df = remove_non_numerics_in_numerics(df, numeric_columns)

### One-Hot Encoding

In [37]:
len(df.columns)

33

In [38]:
df = pd.get_dummies(df, columns=['Raça', 'Sexo', 'Orientação Sexual', 'Gênero', 'Turno', 'Origem escolar', 'Em seu endereço atual, como você mora?', 'Condiçõees de moradia familiar', 'Região da moradia', 'Abastecimento de água', 'Material de construção da moradia', 'Saneamento', 'Qual o grau de parentesco da pessoa que chefia a família?', 'Qual a origem da renda principal de sua família?', 'Como você acessa os serviços de saúde', 'Como você acessou à educação básica (Ensino Fundamental)', 'Como você acessa/acessou à educação básica (Ensino Médio)'], drop_first=False)

In [39]:
len(df.columns)

108

### Transformando "Sim" em **True** e "Não" em **False**.

In [40]:
df.replace({'Sim': True, 'Não': False}, inplace=True)

C:\Users\henri\AppData\Local\Temp\ipykernel_10384\4059199881.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Sim': True, 'Não': False}, inplace=True)


### Removendo linhas com valores negativos

In [41]:
numeric_cols = df.select_dtypes(include='number')

mask = (numeric_cols < 0).any(axis=1)
print(df[mask])
df = df[~mask]

       Data de nascimento  Período  Quantidade de disciplinas no período  \
746                  23.0      4.0                                   7.0   
1078                 23.0      2.0                                   3.0   
1198                 23.0      2.0                                   9.0   
2029                 36.0      3.0                                   8.0   
2259                 25.0      1.0                                   0.0   
2970                 32.0      1.0                                   3.0   
6033                 41.0      1.0                                   6.0   
7525                 21.0      4.0                                   0.0   
7707                 29.0      4.0                                   6.0   
11088                24.0      1.0                                   5.0   
13271                34.0      1.0                                   3.0   
15296                19.0      3.0                                   0.0   
16602       

In [42]:
del numeric_cols
del mask

### Remoção de outliers

In [43]:
def show_min_max_columns(df):
    df_numerico = df.select_dtypes(include=['number'])

    # Obter valores máximos e mínimos para cada coluna numérica
    summary = df_numerico.agg(['max', 'min'])

    print("Resumo dos valores máximos e mínimos:")
    for coluna in df_numerico.columns:
        max_val = summary.loc['max', coluna]
        min_val = summary.loc['min', coluna]
        print(f"{coluna}: max: {max_val}, min: {min_val}")

In [44]:
show_min_max_columns(df)

Resumo dos valores máximos e mínimos:
Data de nascimento: max: 72.0, min: 2.0
Período: max: 20222.0, min: 0.0
Quantidade de disciplinas no período: max: 118.0, min: 0.0
Quantas pessoas compõem o seu núcleo familiar?: max: 10.0, min: 1.0
Você possui filhos maiores que 6 anos: max: 7.0, min: 0.0
Você possui filhos entre 0 e 6 anos: max: 4.0, min: 0.0
Renda bruta familiar: max: 56580.0, min: 0.0
Renda per capita: max: 11316.0, min: 0.0
Alimentação no ambiente escolar: max: 900.0, min: 0.0


In [45]:
outlier_treshold = 8

In [46]:
def remove_outliers(df, outlier_treshold=3):
    # Copia o DataFrame original
    df_clean = df.copy()
    
    # Seleciona apenas as colunas numéricas
    numeric_columns = df_clean.select_dtypes(include=['number']).columns
    
    # Itera pelas colunas numéricas e remove outliers
    for coluna in numeric_columns:
        mean = df_clean[coluna].mean()
        std_dev = df_clean[coluna].std()
        lower_bound = mean - outlier_treshold * std_dev
        upper_bound = mean + outlier_treshold * std_dev
        
        # Filtra o DataFrame removendo as linhas que são outliers
        df_clean = df_clean[(df_clean[coluna] >= lower_bound) & (df_clean[coluna] <= upper_bound)]
    
    return df_clean

In [47]:
df = remove_outliers(df, outlier_treshold)

In [48]:
del outlier_treshold

In [49]:
show_min_max_columns(df)

Resumo dos valores máximos e mínimos:
Data de nascimento: max: 72.0, min: 2.0
Período: max: 2024.0, min: 0.0
Quantidade de disciplinas no período: max: 55.0, min: 0.0
Quantas pessoas compõem o seu núcleo familiar?: max: 10.0, min: 1.0
Você possui filhos maiores que 6 anos: max: 5.0, min: 0.0
Você possui filhos entre 0 e 6 anos: max: 2.0, min: 0.0
Renda bruta familiar: max: 12400.0, min: 0.0
Renda per capita: max: 3059.56, min: 0.0
Alimentação no ambiente escolar: max: 605.0, min: 0.0


### Transformando valores **NaN(Not a Number)/Null(Nulo)/NA(Not Available)** em **-1**

In [50]:
df[df.isna().any(axis=1)]

,Data de nascimento,Período,Quantidade de disciplinas no período,"É cotista por renda inferior a 1,5 salário mínimo?",Moradia estudantil,Mudou de endereço para estudar no IFPE,Quantas pessoas compõem o seu núcleo familiar?,Você possui filhos maiores que 6 anos,Você possui filhos entre 0 e 6 anos,Recebe algum tipo de bolsa estudantil,...,Como você acessa os serviços de saúde_Público e privado,Como você acessa os serviços de saúde_Sistema Público de Saúde,Como você acessou à educação básica (Ensino Fundamental)_Bolsista de escola privada,Como você acessou à educação básica (Ensino Fundamental)_Escola privada,Como você acessou à educação básica (Ensino Fundamental)_Escola pública,Como você acessou à educação básica (Ensino Fundamental)_Pública e privada,Como você acessa/acessou à educação básica (Ensino Médio)_Bolsista de escola privada,Como você acessa/acessou à educação básica (Ensino Médio)_Escola privada,Como você acessa/acessou à educação básica (Ensino Médio)_Escola pública,Como você acessa/acessou à educação básica (Ensino Médio)_Pública e privada


In [51]:
df.columns[df.isna().any(axis=0)]

Index([], dtype='object')

In [52]:
df = df.fillna(-1.0).astype('float64')

### Removendo quaisquer valores que não sejam dos tipos: int, float, bool, str

In [53]:
def remove_invalid_rows(df):
    # Define a função para verificar se um valor é válido
    def is_valid(value):
        if isinstance(value, (int, float, bool, str)):
            return True
        return False
    
    # Aplicar a função para cada valor do DataFrame
    valid_mask = df.map(is_valid)
    
    # Verificar se todas as colunas em cada linha têm valores válidos
    valid_rows = valid_mask.all(axis=1)
    
    # Filtrar o DataFrame para manter apenas as linhas válidas
    df_cleaned = df[valid_rows]
    
    return df_cleaned

In [54]:
df = remove_invalid_rows(df)

## Resultado final

In [55]:
df.reset_index(drop=True, inplace=True)

In [56]:
df

,Data de nascimento,Período,Quantidade de disciplinas no período,"É cotista por renda inferior a 1,5 salário mínimo?",Moradia estudantil,Mudou de endereço para estudar no IFPE,Quantas pessoas compõem o seu núcleo familiar?,Você possui filhos maiores que 6 anos,Você possui filhos entre 0 e 6 anos,Recebe algum tipo de bolsa estudantil,...,Como você acessa os serviços de saúde_Público e privado,Como você acessa os serviços de saúde_Sistema Público de Saúde,Como você acessou à educação básica (Ensino Fundamental)_Bolsista de escola privada,Como você acessou à educação básica (Ensino Fundamental)_Escola privada,Como você acessou à educação básica (Ensino Fundamental)_Escola pública,Como você acessou à educação básica (Ensino Fundamental)_Pública e privada,Como você acessa/acessou à educação básica (Ensino Médio)_Bolsista de escola privada,Como você acessa/acessou à educação básica (Ensino Médio)_Escola privada,Como você acessa/acessou à educação básica (Ensino Médio)_Escola pública,Como você acessa/acessou à educação básica (Ensino Médio)_Pública e privada
0,21.0,4.0,5.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,21.0,5.0,7.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,21.0,2.0,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,20.0,1.0,7.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,19.0,3.0,6.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,20.0,5.0,5.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12218,22.0,2.0,7.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12219,34.0,4.0,4.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
12220,26.0,9.0,7.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [57]:
df.to_excel('../data/cleaned_datas/cleaned_data.xlsx', index=False)